## Orientações:

1. Crie as pastas executando o código em "Kernel", na aba superior dessa tela em seguida clique em "Restart & Run All".

2. Coloque as bases nas pastas de acordo com a origem.

3. Clique novamente em "Kernel" na aba superior dessa tela, em seguida clique em "Restart & Run All".

4. Após alguns segundos as bases estaram salvas e tratadas numa pasta chamada "Saída", no mesmo local desse arquivo.

5. Aparecerá uma mensagem (pop-up), se tudo tiver ocorrido bem.

#### Obs.: Não altere nada no código, somente se tiver certeza do que estiver fazendo.

## Bibliotecas necessárias:

In [ ]:
import importlib

biblioteca = ['pandas', 'os']

for i in biblioteca:
    try:
        importlib.import_module(i)
        print(f"A biblioteca {i} já está instalada.")
    except ImportError:
        print(f"A biblioteca {i} não está instalada. Instalando agora...")
        !pip install {i}
        print(f"A biblioteca {i} foi instalada com sucesso.")

## Funções:

In [ ]:
from IPython.display import display, Javascript
from datetime import datetime
import pandas as pd
import os

def popupPositivo():
    display (Javascript("""
    require(
        ["base/js/dialog"], 
        function(dialog) {
            dialog.modal({
                title: 'Automatização das Bases by JV Pinheiro',
                body: 'Alterações feitas e arquivos .CSV salvos com sucesso.',
                buttons: {
                    'Clique aqui!': {}
                }
            });
        }
    );
    """))
    
def popupNegativo():
    display (Javascript("""
    require(
        ["base/js/dialog"], 
        function(dialog) {
            dialog.modal({
                title: 'Automatização das Bases by JV Pinheiro',
                body: 'As pastas estão vazia ou ocorreu algum erro!',
                buttons: {
                    'Sair': {}
                }
            });
        }
    );
    """))

def tratamentoNina (df):
    # Formatação da data
    df['Data'] = pd.to_datetime(df['Data'], format="%d/%m/%Y %H:%M")
    df['Data'] = df['Data'].dt.strftime("%d/%m/%Y")
    
    # Remoção de um único (DDD +) telefone
    df['Telefone'] = df['Telefone'].astype(str).str[-11:]
    
    # Remoção de espaços em branco no Telefone e Telefones sem DDD
    df.dropna(subset = ['Telefone'])
    df['Telefone'] = df['Telefone'].str.strip()
    df = df[(df['Telefone'].str.len() >= 10) & (df['Telefone'] != '55') & (df['Telefone'] != '99999999999')]
    df['Telefone'] = pd.to_numeric(df['Telefone'], errors='coerce')

    # Remoção de duplicatas com base no telefone
    df = df.drop_duplicates(subset='Telefone')

    return df


def tratamentoRDC (df):
    # Tratamento da data
    df['DATA_INADIMPLENCIA'] = pd.to_datetime(df['DATA_INADIMPLENCIA'])
    df['DATA_INADIMPLENCIA'] = df['DATA_INADIMPLENCIA'].dt.strftime('%d/%m/%Y')

    # Tratamento do telefone (transforma em string e salva os 11 últimos caracteres)
    df['TELEFONE'] = df['TELEFONE'].astype(str)
    df['TELEFONE'] = df['TELEFONE'].str[-11:]

    # Remove duplicatas com base no telefone
    df = df.drop_duplicates(subset='TELEFONE')

    # Remove espaços vazios e número sem DDD
    df = df[(df['TELEFONE'] != '') & (df['TELEFONE'].str.len() >= 10) & (df['TELEFONE'] != '55') & (df['TELEFONE'] != '99999999999')]
    df['TELEFONE'] = pd.to_numeric(df['TELEFONE'], errors='coerce')
    
    # Alterar CPF para tipo numérico
    df['NR_CPF'] = pd.to_numeric(df['NR_CPF'], errors='coerce')
    
    return df

def tratamentoEmail (df):
    # Formatação da data
    df['Data'] = pd.to_datetime(df['Data'])
    df['Data'] = df['Data'].dt.strftime('%d/%m/%Y')

    # Tratamento do telefone
    df['Celular'] = df['Celular'].astype(str)
    df['Celular'] = df['Celular'].str[-11:]

    # Remoção de duplicatas com base na coluna telefone
    df = df.drop_duplicates(subset='Celular')

    # Filtro nas linhas com base no comprimento do número de telefone
    df = df[(df['Celular'] != '') & (df['Celular'].str.len() >= 10) & (df['Celular'] != '55') & (df['Celular'] != '99999999999')]
    df['Celular'] = pd.to_numeric(df['Celular'], errors='coerce')
    
    # Alterar CPF para tipo numérico
    df['NR_CPF'] = pd.to_numeric(df['NR_CPF'], errors='coerce')
    
    return df

def tratamentoConfirma (df):
    # Formatação da data
    df['Date (Data_EXECUCAO)'] = pd.to_datetime(df['Date (Data_EXECUCAO)'])
    df['Date (Data_EXECUCAO)'] = df['Date (Data_EXECUCAO)'].dt.strftime('%d/%m/%Y')

    # Tratamento do telefone
    df['TEL_CELULAR'] = df['TEL_CELULAR'].astype(str)
    df['TEL_CELULAR'] = df['TEL_CELULAR'].str[-11:]

    # Remoção de duplicatas com base na coluna 'Telefone'
    df = df.drop_duplicates(subset='TEL_CELULAR')

    # Filtro nas linhas com base no comprimento do número de telefone
    df = df[(df['TEL_CELULAR'] != '') & (df['TEL_CELULAR'].str.len() >= 10 & (df['TEL_CELULAR'] != '55') & (df['TEL_CELULAR'] != '99999999999'))]
    df['TEL_CELULAR'] = pd.to_numeric(df['TEL_CELULAR'], errors='coerce')
    
    # Alterar CPF para tipo numérico
    df['NR_CPF'] = pd.to_numeric(df['NR_CPF'], errors='coerce')
    
    return df

def tratamentoOdonto (df):
    # Tratamento do telefone
    df['TEL_CELULAR'] = df['TEL_CELULAR'].astype(str)
    df['TEL_CELULAR'] = df['TEL_CELULAR'].str[-11:]

    # Remoção de duplicatas com base na coluna 'Telefone'
    df = df.drop_duplicates(subset='TEL_CELULAR')

    # Filtro nas linhas com base no comprimento do número de telefone
    df = df[(df['TEL_CELULAR'] != '') & (df['TEL_CELULAR'].str.len() >= 10 & (df['TEL_CELULAR'] != '55') & (df['TEL_CELULAR'] != '99999999999'))]
    df['TEL_CELULAR'] = pd.to_numeric(df['TEL_CELULAR'], errors='coerce')
    
    # Alterar CPF para tipo numérico
    df['NR_CPF'] = pd.to_numeric(df['NR_CPF'], errors='coerce')
    
    return df

def tratamentoLittleCar (df):
    # Formatação da data
    df['DT_ACESSO'] = pd.to_datetime(df['DT_ACESSO'])
    df['DT_ACESSO'] = df['DT_ACESSO'].dt.strftime('%d/%m/%Y')

    # Tratamento do telefone
    df['CONTATO'] = df['CONTATO'].astype(str)
    df['CONTATO'] = df['CONTATO'].str[-11:]

    # Remoção de duplicatas com base na coluna 'Telefone'
    df = df.drop_duplicates(subset='CONTATO')

    # Filtro nas linhas com base no comprimento do número de telefone
    df = df[(df['CONTATO'] != '') & (df['CONTATO'].str.len() >= 10 & (df['CONTATO'] != '55') & (df['CONTATO'] != '99999999999'))]
    df['CONTATO'] = pd.to_numeric(df['CONTATO'], errors='coerce')
    
    # Alterar CPF para tipo numérico
    df['CPF'] = pd.to_numeric(df['CPF'], errors='coerce')
    
    return df

## Para Nina (Faltas):

In [ ]:
# Carrega o arquivo Excel em um DataFrame e verifica se existe mais de um arquivo
caminho_pasta = 'Entrada/Nina/FALTAS'
if not os.path.exists(caminho_pasta):
    os.makedirs(caminho_pasta)
arquivos_na_pasta = os.listdir(caminho_pasta)
colunas_para_manter = ['Data', 'Cliente', 'Telefone', 'Status']

lista = []    
if len(arquivos_na_pasta) == 0:
    print ("A pasta está vazia!")
elif len(arquivos_na_pasta) == 1:
    df = pd.read_excel(caminho_pasta + '/' + arquivos_na_pasta[0], usecols=colunas_para_manter, skiprows=1)
elif len(arquivos_na_pasta) > 1:
    for i in arquivos_na_pasta:
        x = pd.read_excel(caminho_pasta + '/' + i, usecols=colunas_para_manter, skiprows=1)
        lista.append(x)
    df = pd.concat(lista)
    
#Verifica se a pasta Saída existe e salva arquivo na pasta
pasta_destino = 'Saída'
if not os.path.exists(pasta_destino):
    os.makedirs(pasta_destino)

if len(arquivos_na_pasta) >= 1:
    nome_arquivo_csv = os.path.join(pasta_destino, 'Absenteísmos-DIAMês.csv')
    tratamentoNina(df).to_csv(nome_arquivo_csv, encoding='utf-16', index=False)

In [ ]:
#Tratamento erro pós-importação - CARRINHO
if os.path.exists('Saída/Absenteísmos-DIAMês.csv'):
    df = pd.read_csv('Saída/Absenteísmos-DIAMês.csv', encoding='utf-16')
    df.to_csv(nome_arquivo_csv, encoding='utf-8', index=False)

## Para Nina (CRIADOS):

In [ ]:
# Carrega o arquivo Excel em um DataFrame e verifica se existe mais de um arquivo
caminho_pasta = 'Entrada/Nina/CRIADOS'
if not os.path.exists(caminho_pasta):
    os.makedirs(caminho_pasta)
arquivos_na_pasta = os.listdir(caminho_pasta)
colunas_para_manter = ['Data', 'Cliente', 'Telefone', 'Status']

lista = []    
if len(arquivos_na_pasta) == 0:
    print ("A pasta está vazia!")
elif len(arquivos_na_pasta) == 1:
    df = pd.read_excel(caminho_pasta + '/' + arquivos_na_pasta[0], usecols=colunas_para_manter, skiprows=1)
elif len(arquivos_na_pasta) > 1:
    for i in arquivos_na_pasta:
        x = pd.read_excel(caminho_pasta + '/' + i, usecols=colunas_para_manter, skiprows=1)
        lista.append(x)
    df = pd.concat(lista)

#Verifica se a pasta Saída existe e salva arquivo na pasta
pasta_destino = 'Saída'
if not os.path.exists(pasta_destino):
    os.makedirs(pasta_destino)

if len(arquivos_na_pasta) >= 1:   
    nome_arquivo_csv = os.path.join(pasta_destino, 'PendentesNina_DIA.MÊS.csv')
    tratamentoNina(df).to_csv(nome_arquivo_csv, encoding='utf-16', index=False) 

In [ ]:
#Tratamento erro pós-importação - CARRINHO
if os.path.exists('Saída/PendentesNina_DIA.MÊS.csv'):
    df = pd.read_csv('Saída/PendentesNina_DIA.MÊS.csv', encoding='utf-16')
    df.to_csv(nome_arquivo_csv, encoding='utf-8', index=False)

## Para RDC:

In [ ]:
# Carrega o arquivo Excel em um DataFrame e verifica se existe mais de um arquivo
caminho_pasta = 'Entrada/RDC'
if not os.path.exists(caminho_pasta):
    os.makedirs(caminho_pasta)
arquivos_na_pasta = os.listdir(caminho_pasta)
colunas_para_manter = ['DATA_INADIMPLENCIA', 'NM_TITULAR', 'CD_PESSOA_FISICA (right)', 'NR_CPF', 'TELEFONE', 'FORMA_PAGAMENTO', 'QTD_FATURAS_PENDENTES (right)', 'Inadimplência em dias', 'Range Inadimplência']

lista = []    
if len(arquivos_na_pasta) == 0:
    print ("A pasta está vazia!")
elif len(arquivos_na_pasta) == 1:
    df = pd.read_excel(caminho_pasta + '/' + arquivos_na_pasta[0], usecols=colunas_para_manter)
elif len(arquivos_na_pasta) > 1:
    for i in arquivos_na_pasta:
        x = pd.read_excel(caminho_pasta + '/' + i, usecols=colunas_para_manter)
        lista.append(x)
    df = pd.concat(lista)

#Verifica se a pasta Saída existe e salva arquivo na pasta
pasta_destino = 'Saída'
if not os.path.exists(pasta_destino):
    os.makedirs(pasta_destino)

if len(arquivos_na_pasta) >= 1:
    nome_arquivo_csv = os.path.join(pasta_destino, 'Base RDC (Inadimplentes) - dia.mês.csv')
    tratamentoRDC(df).to_csv(nome_arquivo_csv, encoding='utf-16', index=False) 

In [ ]:
#Tratamento erro pós-importação - CARRINHO
if os.path.exists('Saída/Base RDC (Inadimplentes) - dia.mês.csv'):
    df = pd.read_csv('Saída/Base RDC (Inadimplentes) - dia.mês.csv', encoding='utf-16')
    df.to_csv(nome_arquivo_csv, encoding='utf-8', index=False)

## Para E-mail (PEP 2 dias):

In [ ]:
# Carrega o arquivo Excel em um DataFrame e verifica se existe mais de um arquivo
caminho_pasta = 'Entrada/E-mail/PEP 2 Dias'
if not os.path.exists(caminho_pasta):
    os.makedirs(caminho_pasta)
arquivos_na_pasta = os.listdir(caminho_pasta)
colunas_para_manter = ['Paciente', 'NR_CPF', 'Data', 'Estabelecimento', 'Celular', 'DS_OBSERVACAO']

lista = []    
if len(arquivos_na_pasta) == 0:
    print ("A pasta está vazia!")
elif len(arquivos_na_pasta) == 1:
    df = pd.read_excel(caminho_pasta + '/' + arquivos_na_pasta[0], usecols=colunas_para_manter, skiprows=4)
elif len(arquivos_na_pasta) > 1:
    for i in arquivos_na_pasta:
        x = pd.read_excel(caminho_pasta + '/' + i, usecols=colunas_para_manter, skiprows=4)
        lista.append(x)
    df = pd.concat(lista)

#Verifica se a pasta Saída existe e salva arquivo na pasta
pasta_destino = 'Saída'
if not os.path.exists(pasta_destino):
    os.makedirs(pasta_destino)

if len(arquivos_na_pasta) >= 1:
    nome_arquivo_csv = os.path.join(pasta_destino, 'PEP 2D DiaMês.csv')
    tratamentoEmail(df).to_csv(nome_arquivo_csv, encoding='utf-16', index=False) 

In [ ]:
#Tratamento erro pós-importação - CARRINHO
if os.path.exists('Saída/PEP 2D DiaMês.csv'):
    df = pd.read_csv('Saída/PEP 2D DiaMês.csv', encoding='utf-16')
    df.to_csv(nome_arquivo_csv, encoding='utf-8', index=False)

## Para E-mail (PEP 7 dias):

In [ ]:
# Carrega o arquivo Excel em um DataFrame e verifica se existe mais de um arquivo
caminho_pasta = 'Entrada/E-mail/PEP 7 Dias'
if not os.path.exists(caminho_pasta):
    os.makedirs(caminho_pasta)
arquivos_na_pasta = os.listdir(caminho_pasta)
colunas_para_manter = ['Paciente', 'NR_CPF', 'Data', 'Estabelecimento', 'Celular', 'DS_OBSERVACAO']

lista = []    
if len(arquivos_na_pasta) == 0:
    print ("A pasta está vazia!")
elif len(arquivos_na_pasta) == 1:
    df = pd.read_excel(caminho_pasta + '/' + arquivos_na_pasta[0], usecols=colunas_para_manter, skiprows=4)
elif len(arquivos_na_pasta) > 1:
    for i in arquivos_na_pasta:
        x = pd.read_excel(caminho_pasta + '/' + i, usecols=colunas_para_manter, skiprows=4)
        lista.append(x)
    df = pd.concat(lista)

#Verifica se a pasta Saída existe e salva arquivo na pasta
pasta_destino = 'Saída'
if not os.path.exists(pasta_destino):
    os.makedirs(pasta_destino)

if len(arquivos_na_pasta) >= 1:
    nome_arquivo_csv = os.path.join(pasta_destino, 'PEP 7D DiaMês.csv')
    tratamentoEmail(df).to_csv(nome_arquivo_csv, encoding='utf-16', index=False) 

In [ ]:
#Tratamento erro pós-importação - CARRINHO
if os.path.exists('Saída/PEP 7D DiaMês.csv'):
    df = pd.read_csv('Saída/PEP 7D DiaMês.csv', encoding='utf-16')
    df.to_csv(nome_arquivo_csv, encoding='utf-8', index=False)

## Para E-mail (PEP 15 dias):

In [ ]:
# Carrega o arquivo Excel em um DataFrame e verifica se existe mais de um arquivo
caminho_pasta = 'Entrada/E-mail/PEP 15 Dias'
if not os.path.exists(caminho_pasta):
    os.makedirs(caminho_pasta)
arquivos_na_pasta = os.listdir(caminho_pasta)
colunas_para_manter = ['Paciente', 'NR_CPF', 'Data', 'Estabelecimento', 'Celular', 'DS_OBSERVACAO']

lista = []    
if len(arquivos_na_pasta) == 0:
    print ("A pasta está vazia!")
elif len(arquivos_na_pasta) == 1:
    df = pd.read_excel(caminho_pasta + '/' + arquivos_na_pasta[0], usecols=colunas_para_manter, skiprows=4)
elif len(arquivos_na_pasta) > 1:
    for i in arquivos_na_pasta:
        x = pd.read_excel(caminho_pasta + '/' + i, usecols=colunas_para_manter, skiprows=4)
        lista.append(x)
    df = pd.concat(lista)

#Verifica se a pasta Saída existe e salva arquivo na pasta
pasta_destino = 'Saída'
if not os.path.exists(pasta_destino):
    os.makedirs(pasta_destino)

if len(arquivos_na_pasta) >= 1:
    nome_arquivo_csv = os.path.join(pasta_destino, 'PEP 15D DiaMês.csv')
    tratamentoEmail(df).to_csv(nome_arquivo_csv, encoding='utf-16', index=False) 

In [ ]:
#Tratamento erro pós-importação - CARRINHO
if os.path.exists('Saída/PEP 15D DiaMês.csv'):
    df = pd.read_csv('Saída/PEP 15D DiaMês.csv', encoding='utf-16')
    df.to_csv(nome_arquivo_csv, encoding='utf-8', index=False)

## Para E-mail (Confirmação):

In [ ]:
# Carrega o arquivo Excel em um DataFrame e verifica se existe mais de um arquivo
caminho_pasta = 'Entrada/E-mail/Confirmação'
if not os.path.exists(caminho_pasta):
    os.makedirs(caminho_pasta)
arquivos_na_pasta = os.listdir(caminho_pasta)
colunas_para_manter = ['NM_PESSOA_FISICA', 'NR_CPF', 'TEL_CELULAR','Date (Data_EXECUCAO)', 'NM_ESTABELECIMENTO']

lista = []    
if len(arquivos_na_pasta) == 0:
    print ("A pasta está vazia!")
elif len(arquivos_na_pasta) == 1:
    df = pd.read_excel(caminho_pasta + '/' + arquivos_na_pasta[0], usecols=colunas_para_manter, skiprows=2)
elif len(arquivos_na_pasta) > 1:
    for i in arquivos_na_pasta:
        x = pd.read_excel(caminho_pasta + '/' + i, usecols=colunas_para_manter, skiprows=2)
        lista.append(x)
    df = pd.concat(lista)

#Verifica se a pasta Saída existe e salva arquivo na pasta
pasta_destino = 'Saída'
if not os.path.exists(pasta_destino):
    os.makedirs(pasta_destino)

if len(arquivos_na_pasta) >= 1:
    nome_arquivo_csv = os.path.join(pasta_destino, 'REMARCACAO_DIA.MÊS.csv')
    tratamentoConfirma(df).to_csv(nome_arquivo_csv, encoding='utf-16', index=False)

In [ ]:
#Tratamento erro pós-importação - CARRINHO
if os.path.exists('Saída/REMARCACAO_DIA.MÊS.csv'):
    df = pd.read_csv('Saída/REMARCACAO_DIA.MÊS.csv', encoding='utf-16')
    df.to_csv(nome_arquivo_csv, encoding='utf-8', index=False)

## Para E-mail (Absenteísmo):

In [ ]:
# Carrega o arquivo Excel em um DataFrame e verifica se existe mais de um arquivo
caminho_pasta = 'Entrada/E-mail/Absenteísmo'
if not os.path.exists(caminho_pasta):
    os.makedirs(caminho_pasta)
arquivos_na_pasta = os.listdir(caminho_pasta)
colunas_para_manter = ['NM_PESSOA_FISICA', 'NR_CPF', 'TEL_CELULAR', 'NM_ESTABELECIMENTO', 'Date (Data_EXECUCAO)', 'DS_STATUS_AGENDA']

lista = []    
if len(arquivos_na_pasta) == 0:
    print ("A pasta está vazia!")
elif len(arquivos_na_pasta) == 1:
    df = pd.read_excel(caminho_pasta + '/' + arquivos_na_pasta[0], usecols=colunas_para_manter, skiprows=3)
elif len(arquivos_na_pasta) > 1:
    for i in arquivos_na_pasta:
        x = pd.read_excel(caminho_pasta + '/' + i, usecols=colunas_para_manter, skiprows=3)
        lista.append(x)
    df = pd.concat(lista)

#Verifica se a pasta Saída existe e salva arquivo na pasta
pasta_destino = 'Saída'
if not os.path.exists(pasta_destino):
    os.makedirs(pasta_destino)

if len(arquivos_na_pasta) >= 1:
    nome_arquivo_csv = os.path.join(pasta_destino, 'Absenteísmos2-DiaMês.csv')
    tratamentoConfirma(df).to_csv(nome_arquivo_csv, encoding='utf-16', index=False)

In [ ]:
#Tratamento erro pós-importação - CARRINHO
if os.path.exists('Saída/Absenteísmos2-DiaMês.csv'):
    df = pd.read_csv('Saída/Absenteísmos2-DiaMês.csv', encoding='utf-16')
    df.to_csv(nome_arquivo_csv, encoding='utf-8', index=False)

## Para Odonto:

In [ ]:
# Carrega o arquivo Excel em um DataFrame e verifica se existe mais de um arquivo
caminho_pasta = 'Entrada/Odonto'
if not os.path.exists(caminho_pasta):
    os.makedirs(caminho_pasta)
arquivos_na_pasta = os.listdir(caminho_pasta)
colunas_para_manter = ['NM_ESTABELECIMENTO', 'NR_CPF', 'NM_PESSOA_FISICA', 'TEL_CELULAR']

lista = []    
if len(arquivos_na_pasta) == 0:
    print ("A pasta está vazia!")
elif len(arquivos_na_pasta) == 1:
    df = pd.read_excel(caminho_pasta + '/' + arquivos_na_pasta[0], usecols=colunas_para_manter)        
elif len(arquivos_na_pasta) > 1:
    for i in arquivos_na_pasta:
        x = pd.read_excel(caminho_pasta + '/' + i, usecols=colunas_para_manter)
        lista.append(x)
    df = pd.concat(lista)

#Verifica se a pasta Saída existe e salva arquivo na pasta
pasta_destino = 'Saída'
if not os.path.exists(pasta_destino):
    os.makedirs(pasta_destino)

if len(arquivos_na_pasta) >= 1:
    nome_arquivo_csv = os.path.join(pasta_destino, 'Odonto dia.mês.csv')
    tratamentoOdonto(df).to_csv(nome_arquivo_csv, encoding='utf-16', index=False) 

In [ ]:
#Tratamento erro pós-importação - CARRINHO
if os.path.exists('Saída/Odonto dia.mês.csv'):
    df = pd.read_csv('Saída/Odonto dia.mês.csv', encoding='utf-16')
    df.to_csv(nome_arquivo_csv, encoding='utf-8', index=False)

## Para Carrinho:

In [ ]:
# Carrega o arquivo Excel em um DataFrame e verifica se existe mais de um arquivo
caminho_pasta = 'Entrada/Carrinho'
if not os.path.exists(caminho_pasta):
    os.makedirs(caminho_pasta)
arquivos_na_pasta = os.listdir(caminho_pasta)
colunas_para_manter = ['DT_ACESSO', 'UNIDADE', 'CPF', 'NOME', 'STATUS', 'CONTATO', 'ESPECIALIDADE']

lista = []    
if len(arquivos_na_pasta) == 0:
    print ("A pasta está vazia!")
elif len(arquivos_na_pasta) == 1:
    df = pd.read_excel(caminho_pasta + '/' + arquivos_na_pasta[0], usecols=colunas_para_manter)
elif len(arquivos_na_pasta) > 1:
    for i in arquivos_na_pasta:
        x = pd.read_excel(caminho_pasta + '/' + i, usecols=colunas_para_manter)
        lista.append(x)
    df = pd.concat(lista)

#Verifica se a pasta Saída existe e salva arquivo na pasta
pasta_destino = 'Saída'
if not os.path.exists(pasta_destino):
    os.makedirs(pasta_destino)

if len(arquivos_na_pasta) >= 1:
    nome_arquivo_csv = os.path.join(pasta_destino, 'Carrinho Abandonado DiaMês.csv')
    tratamentoLittleCar(df).to_csv(nome_arquivo_csv, encoding='utf-16', index=False)

In [ ]:
#Tratamento erro pós-importação - CARRINHO
if os.path.exists('Saída/Carrinho Abandonado DiaMês.csv'):
    df = pd.read_csv('Saída/Carrinho Abandonado DiaMês.csv', encoding='utf-16')
    df['UNIDADE'] = df['UNIDADE'].str.replace('�','')
    df['ESPECIALIDADE'] = df['ESPECIALIDADE'].str.replace('�','í')
    df['NOME'] = df['NOME'].str.replace('�','_')
    df.to_csv(nome_arquivo_csv, encoding='utf-8', index=False)

## Para Checkup 365D:

In [ ]:
# Carrega o arquivo Excel em um DataFrame e verifica se existe mais de um arquivo
caminho_pasta = 'Entrada/365D'
if not os.path.exists(caminho_pasta):
    os.makedirs(caminho_pasta)
arquivos_na_pasta = os.listdir(caminho_pasta)
colunas_para_manter = ['Date (Data_EXECUCAO)', 'TEL_CELULAR', 'NM_PESSOA_FISICA', 'DS_PROCEDIMENTO', 'NR_CPF']

lista = []    
if len(arquivos_na_pasta) == 0:
    print ("A pasta está vazia!")
elif len(arquivos_na_pasta) == 1:
    df = pd.read_excel(caminho_pasta + '/' + arquivos_na_pasta[0], usecols=colunas_para_manter)
elif len(arquivos_na_pasta) > 1:
    for i in arquivos_na_pasta:
        x = pd.read_excel(caminho_pasta + '/' + i, usecols=colunas_para_manter)
        lista.append(x)
    df = pd.concat(lista)

#Verifica se a pasta Saída existe e salva arquivo na pasta
pasta_destino = 'Saída'
if not os.path.exists(pasta_destino):
    os.makedirs(pasta_destino)

if len(arquivos_na_pasta) >= 1:
    nome_arquivo_csv = os.path.join(pasta_destino, 'Checkup dd.mm.aaaa.csv')
    tratamentoConfirma(df).to_csv(nome_arquivo_csv, encoding='utf-16', index=False) 

In [ ]:
#Tratamento erro pós-importação - CARRINHO
if os.path.exists('Saída/Checkup dd.mm.aaaa.csv'):
    df = pd.read_csv('Saída/Checkup dd.mm.aaaa.csv', encoding='utf-16')
    df.to_csv(nome_arquivo_csv, encoding='utf-8', index=False)

## Para Particulares:

In [ ]:
# Carrega o arquivo Excel em um DataFrame e verifica se existe mais de um arquivo
caminho_pasta = 'Entrada/Particulares'
if not os.path.exists(caminho_pasta):
    os.makedirs(caminho_pasta)
arquivos_na_pasta = os.listdir(caminho_pasta)
colunas_para_manter = ['data', 'codigo_paciente', 'cpf_paciente', 'nome_paciente', 'tipo_agenda', 'descricao_especialidade_agenda', 'nome_estabelecimento', 'telefone_paciente']

lista = []    
if len(arquivos_na_pasta) == 0:
    print ("A pasta está vazia!")
elif len(arquivos_na_pasta) == 1:
    df = pd.read_excel(caminho_pasta + '/' + arquivos_na_pasta[0], usecols=colunas_para_manter, skiprows=4)
elif len(arquivos_na_pasta) > 1:
    for i in arquivos_na_pasta:
        x = pd.read_excel(caminho_pasta + '/' + i, usecols=colunas_para_manter, skiprows=4)
        lista.append(x)
    df = pd.concat(lista)

#Verifica se a pasta Saída existe e salva arquivo na pasta
pasta_destino = 'Saída'
if not os.path.exists(pasta_destino):
    os.makedirs(pasta_destino)

if len(arquivos_na_pasta) >= 1:
    nome_arquivo_csv = os.path.join(pasta_destino, 'Particulares ddmmm.csv')
    tratamentoEmail(df).to_csv(nome_arquivo_csv, encoding='utf-16', index=False)

In [ ]:
#Tratamento erro pós-importação - CARRINHO
if os.path.exists('Saída/Particulares ddmmm.csv'):
    df = pd.read_csv('Saída/Particulares ddmmm.csv', encoding='utf-16')
    df.to_csv(nome_arquivo_csv, encoding='utf-8', index=False)

## Aviso pop-up:

In [ ]:
#Condicional do aviso na Tela
if os.path.exists('Saída/Particulares ddmmm.csv') or os.path.exists('Saída/Checkup dd.mm.aaaa.csv') or os.path.exists('Saída/Carrinho Abandonado DiaMês.csv') or os.path.exists('Saída/Odonto dia.mês.csv') or os.path.exists('Saída/REMARCACAO_DIA.MÊS.csv')or os.path.exists('Saída/PEP 15D DiaMês.csv') or os.path.exists('Saída/PEP 7D DiaMês.csv') or os.path.exists('Saída/PEP 2D DiaMês.csv') or os.path.exists('Saída/PendentesNina_DIA.MÊS.csv') or os.path.exists('Saída/Absenteísmos-DIAMês.csv') or os.path.exists('Saída/Carrinho Abandonado DiaMês.csv') or os.path.exists('Saída/Odonto dia.mês.csv') or os.path.exists('Saída/REMARCACAO_DIA.MÊS.csv')or os.path.exists('Saída/PEP 15D DiaMês.csv') or os.path.exists('Saída/PEP 7D DiaMês.csv') or os.path.exists('Saída/PEP 2D DiaMês.csv') or os.path.exists('Saída/PendentesNina_DIA.MÊS.csv') or os.path.exists('Saída/Absenteísmos-DIAMês.csv') or os.path.exists('Saída/Base RDC (Inadimplentes) - dia.mês.csv'):
    popupPositivo()
    print("Alterações feitas e arquivos .CSV salvos com sucesso.")
else:
    popupNegativo()